In [1]:
import pandas as pd
import numpy as np
import re
import time
import io
import webbrowser
import datetime
import random

import requests as rq
import bs4 as bs4 # beautifulsoup4
import json
from pymongo import MongoClient

from feature_extractor import convert_datetime

In [2]:
queries = ["data%20scientist"]#, "cientista%20de%20dados", "machine%20learning"]
#locations = ["São%20Paulo%2C%20Brazil", "Porto%20Alegre%2C%20Rio%20Grande%20do%20Sul%2C%20Brazil", "europe"]
locations = ["São%20Paulo%2C%20Brazil", "Porto%20Alegre%2C%20Rio%20Grande%20do%20Sul%2C%20Brazil", "europe", "England", "Italy", "Netherlands", "Spain"]
url="https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={query}&location={location}&pageNum=0&start={page}"
#url="https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={query}&location={location}&pageNum=0&f_TP=1%2C2&start={page}"

In [3]:
client = MongoClient()
db = client.linkedin
collection = db.jobIds

In [4]:
headers = {
    "User-Agent" : "PythonRequests",
}

In [16]:
start = 0
for query in queries:
    for location in locations:
        for page in range(0,1):
            urll = url.format(query=query, page=page*25, location=location)
            print(urll)
            response = rq.get(urll, headers=headers)            
            with io.open("./pages/{}_{}_{}.html".format(query,location, page).replace("%20", "_"), "w", encoding="utf-8") as f:
                f.write(response.text)

            parsed = bs4.BeautifulSoup(response.text)
            tags = parsed.findAll("li", class_="result-card")

            time.sleep(2.5)

            print(len(tags))

            if len(tags) < 25:
                break



https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%20scientist&location=São%20Paulo%2C%20Brazil&pageNum=0&f_TP=1%2C2&start=0
25
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%20scientist&location=Porto%20Alegre%2C%20Rio%20Grande%20do%20Sul%2C%20Brazil&pageNum=0&f_TP=1%2C2&start=0
12
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%20scientist&location=europe&pageNum=0&f_TP=1%2C2&start=0
25
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%20scientist&location=England&pageNum=0&f_TP=1%2C2&start=0
25
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%20scientist&location=Italy&pageNum=0&f_TP=1%2C2&start=0
25
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%20scientist&location=Netherlands&pageNum=0&f_TP=1%2C2&start=0
25
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPost

In [21]:
new_documents = 0

for document in os.listdir('./pages'):
    with io.open("./pages/{}".format(document), "r", encoding="utf-8") as inp:
        page_html = inp.read()
        
        print(f"Page {document}:")
        parsed = bs4.BeautifulSoup(page_html)
        
        tags = parsed.findAll("li", class_="result-card")
        for tag in tags:
            item = {'jobId': tag['data-id'], 'loaded': False, 'added' : datetime.datetime.utcnow()}
            alreadyIn = collection.find_one({"jobId": item['jobId']})
            if not alreadyIn:
                print(f"Adding {item['jobId']}")
                new_documents += 1 
                collection.insert_one(item)

print(f"Added {new_documents} new jobs!")

Page data_scientist_England_0.html:
Adding 1932926640
Adding 1933716917
Adding 1935522233
Adding 1087403280
Adding 1928750956
Adding 1909832377
Adding 1909048253
Adding 1933923773
Adding 1909836435
Page data_scientist_europe_0.html:
Adding 1935624667
Adding 1932135058
Adding 1909819877
Adding 1909433877
Adding 1931941551
Adding 1909810723
Adding 1908839545
Adding 1904306589
Adding 1904337941
Adding 1930531602
Adding 1932593136
Adding 1908867737
Adding 1909410264
Adding 1936287688
Adding 1903187997
Page data_scientist_Italy_0.html:
Adding 1909832781
Adding 1909810352
Adding 1905092840
Adding 1907735616
Adding 1935963023
Adding 1935580201
Adding 1935673035
Adding 1932151129
Adding 1905086849
Adding 1931647160
Adding 1907775433
Page data_scientist_Netherlands_0.html:
Adding 1931710890
Adding 1935597063
Adding 1932700508
Adding 1932951913
Adding 1935769667
Adding 1909822645
Adding 1909076032
Adding 1909806363
Adding 1909441657
Adding 1932545891
Adding 1927857000
Adding 1930427252
Adding 19

In [24]:
for job in collection.find({'loaded': False}):
    url = "https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{jobId}"
    urll = url.format(jobId=job['jobId'])
    print(urll)
    response = rq.get(urll)
    with io.open("./jobs/{}.html".format(job['jobId']), "w", encoding="utf-8") as f:
        f.write(response.text)
        job['loaded'] = True
        job['created'] = datetime.datetime.utcnow()
        collection.save(job)
    time.sleep(random.randint(2,5))


https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1933716917
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1935522233
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1087403280
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1928750956
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1909832377
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1909048253
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1933923773
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1909836435
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1935624667
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1932135058
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1909819877
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1909433877
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1931941551
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/1909810723
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/190883

In [25]:
collection.count_documents({'loaded': False})
collection.count_documents({'loaded': True, 'title': None})

80

In [9]:
for i, job in enumerate(collection.find({'loaded': True, 'title': None})):
    jobId = job['jobId']
    with io.open("./jobs/{}.html".format(jobId), "r", encoding="utf-8") as f:
        
        print(f"Job {jobId}:")

        #BS4
        page_html = f.read()
        parsed = bs4.BeautifulSoup(page_html)
        company_a = parsed.findAll("a", class_="topcard__org-name-link")
        if company_a:
            job['company'] = company_a[0].get_text()
        else:
            job['company'] = parsed.findAll("span", class_="topcard__flavor")[0].get_text()
        job['title'] = parsed.findAll("h2", class_="topcard__title")[0].get_text()
        job['city'] = parsed.findAll("span", class_="topcard__flavor topcard__flavor--bullet")[0].contents[0]
        delta_posted = convert_datetime(parsed.findAll("span", class_=["topcard__flavor--metadata posted-time-ago__text", "topcard__flavor--metadata posted-time-ago__text posted-time-ago__text--new"])[0].contents[0])
        job['posted'] = job['created'] - delta_posted
        applicants = parsed.findAll("span", class_="topcard__flavor--metadata topcard__flavor--bullet num-applicants__caption")
        if applicants:
            job['applicants'] = int(applicants[0].get_text().split(' ')[0])
        else:
            job['applicants'] = 10
        job['applicants_per_day'] = job['applicants'] / (delta_posted.days + 1)
        job['description'] = parsed.findAll("div", class_="show-more-less-html__markup show-more-less-html__markup--clamp-after-5")[0].get_text()
        job['description_len'] = len(job['description'])
        job_criterion = parsed.findAll("span", class_="job-criteria__text job-criteria__text--criteria")
        job_header = parsed.findAll("h3", class_="job-criteria__subheader")
        for header, value in zip(job_header, job_criterion):
            job[header.get_text()] = value.get_text()

        collection.save(job)

In [6]:
for job in collection.aggregate([{ "$match": { "label": { "$exists": False } } }, { "$sample": { "size": 20 }}]):
    url = f"https://www.linkedin.com/jobs/view/{job['jobId']}"
    print(job['title'], job['company'])
    webbrowser.open(url)
    job['label'] = int(input())
    assert(job['label'] in [0,1,-1])
    print(job['label'])
    if job['label'] == -1:
        print('Deleting job')
        collection.delete_one({'jobId': job['jobId']})
    else:
        collection.save(job)

Data analyst Affin Hwang Capital
0
Data analyst - Matipay MATIPAY - Mobile Payment and Telemetry system for Vending
0
Meteorological Data Scientist DTN
0
Data Scientist (ASL) Nestlé
0
Analista de Banco de Dados - Ciência de Dados HST CARD TECHNOLOGY
0
Functional Data Analyst Securex
0
SENIOR DATA ENGINEER Omina Technologies
0
Business Data Analyst Vaillant Group
0
Sales Engineer - Brazil Alarm.com
0
Technology Delivery Manager Twipe
0
Sr Backend Developer (Python) Inovation IT Premium Outsourcing
0
Estágio em inteligência (bi) Petlove
0
Junior Data scientist / Planning Process Expert AkzoNobel
0
Data Scientist Rightangled
0
Data Scientist Marks and Spencer
1
Deals - Investigative Analytics - Data Scientist - London, PWC Guardian Jobs
1
NLP Data Scientist PEP Health
1
Data Science Intern StyleSage
0
Consultor de políticas de crédito e modelagem Page Personnel
0
Data Scientist Compass Lexecon
1


In [10]:
print(collection.count_documents({'label' : 1}))
print(collection.count_documents({'label' : 0}))

collection.count_documents({'label' : 1}) + collection.count_documents({'label' : 0})

135
464


599

In [30]:
collection.count_documents({'label' : None})

887

In [27]:
for pin in collection.aggregate([{ "$match": { "label": { "$exists": False } } }, { "$sample": { "size": 100 }}]):
    print(pin['jobId'])

1929998936
1927462021
1897388221
1897967956
1865633769
1828820756
1879666038
1869851010
1875306659
1904055066
1904082120
1907869128
1923284989
1865660361
1928207071
1519956488
1805699370
1888811341
1840740404
1718334914
1894888703
1903926427
1908812796
1826241938
1908690579
1927388015
1924685614
1841944610
1924392079
1745899361
1926690401
1910988623
1922342401
1924006645
1709415926
1921682681
1782841567
1893780920
1926516629
1789135389
1928206872
1927162480
1877930054
1901517734
1338288139
1927978745
1519410583
1910855673
1907230784
1727111489
1924886357
1899042606
1916890579
1892340943
1928743299
1884435598
1923548632
1901038239
1930860525
1905069958
1764181663
1914372928
1902203672
1819063930
1892511470
1918932652
1904582418
1894800387
1879425668
1894813818
1727102879
1924217037
1915866768
1811691646
1895675890
1892679695
1804787596
1921906860
1684919361
1881125522
1819318222
1879479030
1901135268
1882050386
1738600127
1897326663
1888163448
1883149435
1922475558
1922238429
1877473764

In [4]:
import datetime

In [5]:
datetime.datetime.utcnow()

datetime.datetime(2020, 7, 6, 22, 27, 37, 726961)

In [20]:
datetime.datetime.utcnow() - datetime.timedelta(days=6)

datetime.datetime(2020, 6, 30, 22, 40, 34, 287034)